In [1]:
import os
import pandas as pd
import dataframe_image as dfi
from PIL import Image, ImageDraw, ImageFont
from gcd_table_sequential import build_gcd_table_filled

In [2]:
# example from:
#  https://win-vector.com/2024/10/06/dudeneys-remainder-problem/
soln = build_gcd_table_filled(
    508811 - 480608, 
    723217 - 480608,
    verbose=True)

'Initial table: a = 242609, b = 28203'

,r,q,vu
0,242609,,
1,28203,,
2,,,
3,,,
4,,,
5,,,
6,,,
7,,,
8,,,


'build row 2: r[2] = r[0] % r[1], q[2] = r[0] // r[1]'

,r,q,vu
0,242609,,
1,28203,,
2,16985,8,
3,,,
4,,,
5,,,
6,,,
7,,,
8,,,


'build row 3: r[3] = r[1] % r[2], q[3] = r[1] // r[2]'

...

,r,q,vu
0,242609,,
1,28203,,
2,16985,8,
3,11218,1,53
4,5767,1,-35
5,5451,1,18
6,316,1,-17
7,79,17,1
8,0,4,0


'back fill row 2: vu[2] = vu[4] - q[3] * vu[3]'

,r,q,vu
0,242609,,
1,28203,,
2,16985,8,-88
3,11218,1,53
4,5767,1,-35
5,5451,1,18
6,316,1,-17
7,79,17,1
8,0,4,0


'back fill row 1: vu[1] = vu[3] - q[2] * vu[2]'

,r,q,vu
0,242609,,
1,28203,,757
2,16985,8,-88
3,11218,1,53
4,5767,1,-35
5,5451,1,18
6,316,1,-17
7,79,17,1
8,0,4,0


In [3]:

tables = list(soln.attrs['captured_tables'])  # new list, so we can append without side effects

In [4]:
# convert tables to images
for i, t in enumerate(tables):
    file_name = f'dataframe_{i:04d}.png'
    # convert to PNG
    dfi.export(t, file_name, dpi=600)


In [5]:
# add annotation notes
font = ImageFont.load_default(size=50)
text_position = (50, 10)  # x, y coordinates for the text
text_color = (0, 0, 0)
for i, t in enumerate(tables):
    file_name = f'dataframe_{i:04d}.png'
    file_name_out = f'dataframe_annotated_{i:04d}.png'
    note = t.data.attrs['note']
    # add in text or copy image  
    image = Image.open(file_name)
    draw = ImageDraw.Draw(image)
    if note is not None:
        draw.text(text_position, note, fill=text_color, font=font)
    image.save(file_name_out)

Then convert at the command line with:

```
ffmpeg -framerate 0.5 -i dataframe_annotated_%04d.png -c:v libx264 -r 30 -pix_fmt yuv420p RemainderProblemSeq.mp4
```


In [6]:
!ffmpeg -framerate 0.5 -i dataframe_annotated_%04d.png -c:v libx264 -r 30 -pix_fmt yuv420p RemainderProblemSeq.mp4

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.1.2_6 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --

In [7]:
# clean up
for i, t in enumerate(tables):
    file_name = f'dataframe_{i:04d}.png'
    file_name_out = f'dataframe_annotated_{i:04d}.png'
    os.remove(file_name)
    os.remove(file_name_out)